In [18]:
# from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
# from langchain.callbacks import StreamingStdOutCallbackHandler, get_openai_callback
# from langchain.prompts import example_selector
# from langchain.prompts.example_selector.base import BaseExampleSelector
# from langchain.prompts.pipeline import PipelinePromptTemplate
# from langchain.globals import set_llm_cache, set_debug
# from langchain.cache import InMemoryCache, SQLiteCache
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory, ConversationSummaryBufferMemory, ConversationKGMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, load_prompt, MessagesPlaceholder


llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True
)

chain.predict(question="내 이름은 정훈입니다.")




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: 내 이름은 정훈입니다.

> Finished chain.


'안녕하세요 정훈님! 만나서 반갑습니다. 저는 인공지능입니다. 무엇을 도와드릴까요?'

In [19]:
chain.predict(question="나는 용인에 살아요.")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: 내 이름은 정훈입니다.
AI: 안녕하세요 정훈님! 만나서 반갑습니다. 저는 인공지능입니다. 무엇을 도와드릴까요?
Human: 나는 용인에 살아요.

> Finished chain.


'용인은 경기도에 위치한 아름다운 도시입니다. 어떤 도움이 필요하신가요? 관광지 추천이나 현지 정보, 교통 정보 등을 알려드릴 수 있습니다.'

In [20]:
chain.predict(question="내 이름이 뭔가요?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
System: The human introduces themselves as 정훈. The AI greets 정훈 and introduces itself as an artificial intelligence, asking how it can assist.
Human: 나는 용인에 살아요.
AI: 용인은 경기도에 위치한 아름다운 도시입니다. 어떤 도움이 필요하신가요? 관광지 추천이나 현지 정보, 교통 정보 등을 알려드릴 수 있습니다.
Human: 내 이름이 뭔가요?

> Finished chain.


'당신의 이름은 정훈입니다.'

In [21]:
chain.predict(question="나를 어떻게 도와줄수 있나요?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
System: The human introduces themselves as 정훈 and the AI greets 정훈, introducing itself as an artificial intelligence and asking how it can assist. 정훈 responds by saying they live in Yongin.
AI: 용인은 경기도에 위치한 아름다운 도시입니다. 어떤 도움이 필요하신가요? 관광지 추천이나 현지 정보, 교통 정보 등을 알려드릴 수 있습니다.
Human: 내 이름이 뭔가요?
AI: 당신의 이름은 정훈입니다.
Human: 나를 어떻게 도와줄수 있나요?

> Finished chain.


'저는 여러 가지 방법으로 도움을 드릴 수 있습니다. 예를 들어, 여행 관련 정보를 제공하거나 교통 정보를 알려드릴 수 있습니다. 또한 일상 생활에서 필요한 정보나 질문에 대한 답변을 제공할 수도 있습니다. 무엇을 도와드릴까요?'